In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
%matplotlib inline
plt.viridis()
import time
from brian2.units import *
from scipy.spatial.distance import minkowski

In [2]:
def distance(s, t, grid_shape, dimensions):
        '''
        Function that computes distance in a grid of neurons taking into account periodic boundry conditions.

        First, translate source into the center of the grid.
        Second, translate target by the same amount.
        Finally, perform desired distance computation.
        '''
        s = np.asarray(s)
        t = np.asarray(t)
        _grid_size = np.asarray(grid_shape)
        trans = s - (_grid_size // 2)
        s = np.mod(s - trans, _grid_size)
        t = np.mod(t - trans, _grid_size)
        return minkowski(s, t, dimensions)

In [3]:
def generate_rates(s, grid_shape, dimensions):
    '''
    Function that generates an array the same shape as the input layer so that 
    each cell has a value corresponding to the firing rate for the neuron
    at that position.
    '''
    _rates = np.zeros(grid_shape)
    for x, y in np.ndindex(grid_shape):
        _d = distance(s, (x,y), grid_shape=grid_shape, dimensions=dimensions)
        _rates[x, y] = f_base + f_peak * np.e ** (-_d/(2 * sigma_stim**2))
    return _rates * Hz

In [4]:
duration = 100 * ms
d = 2
# Wiring
n = 16
N_layer = n ** 2
S = (n, n)

s = (n//2, n//2)
sigma_form_forward = 2.5
sigma_form_lateral = 1

# Inputs
f_mean = 20 * Hz
f_base = 5 * Hz
f_peak = 152.8  * Hz
sigma_stim = 2
t_stim = 0.02 * second

In [5]:
_chunk = 20 * ms
_dt = .1 * ms
_chunk / _dt

200.0

In [6]:
smth = [[],]

In [7]:
smth.append([3,2,1])

In [8]:
smth

[[], [3, 2, 1]]

The idea is to generate a 2D spike time array (different times for different spike source). To obtain a set a times I need to consider the rate of each spiking source, then generate a poisson series, and convert nonzero indices to times.

In [9]:
def generate_spike_times(s, dt=.1 * ms, chunk = 20 * ms, time_offset=0. * ms, grid_shape=(16,16), dimensions=2):
    rates = generate_rates(s, grid_shape, dimensions)
    spike_times = []
    for _, rate in np.ndenumerate(rates/Hz):
        spikes = np.random.poisson(rate/1000., int(chunk/dt))
        fucking_indices = np.nonzero(spikes)[0]
        fucking_indices_as_times = fucking_indices * dt
        shifted_fucking_indices = fucking_indices_as_times + time_offset
        spike_times.append((( shifted_fucking_indices)/ms).tolist())
    return spike_times

In [16]:
spike_times = generate_spike_times((13,8), time_offset=20*ms, grid_shape=S, dimensions=2)

In [18]:
spike_times[1]

[20.900000000000002,
 21.8,
 22.6,
 23.599999999999998,
 26.9,
 28.200000000000003,
 29.8,
 30.8,
 32.4,
 35.3,
 36.50000000000001,
 39.5]

In [11]:
def give_me_spike_times(N = 256, duration=25*ms, t_stim = 20*ms, dimensions=2):
    spike_times = [[], ] * N
    time_slot = 0
    for time_slot in range(int(duration / t_stim)):
        _sp = generate_spike_times(np.random.randint(0, np.sqrt(N), 2),
                                chunk=t_stim,
                                time_offset=time_slot * t_stim,
                                dimensions=dimensions,
                                  grid_shape=(np.sqrt(N),np.sqrt(N)))
        for index, value in np.ndenumerate(_sp):
            if hasattr(value , '__iter__'):
                for v in value:
                    spike_times[index[0]].append(v)
            else:
                spike_times[index[0]].append(value)
    print "time slot", time_slot
    if not np.isclose(duration % t_stim / ms, 0):
        print duration % t_stim
        _sp = generate_spike_times(np.random.randint(0, 16, 2),
                                chunk=duration % t_stim,
                                time_offset=(time_slot+1) * t_stim,
                                dimensions=dimensions,
                                  grid_shape=(np.sqrt(N),np.sqrt(N)))
        for index, value in np.ndenumerate(_sp):
            if hasattr(value, '__iter__'):
                for v in value:
                    spike_times[index[0]].append(v)
            else:
                spike_times[index[0]].append(value)
    return spike_times

In [12]:
spike_times = give_me_spike_times(4)

WARNING    /home/bogdanp/PhD/phd_venv/lib/python2.7/site-packages/ipykernel/__main__.py:7: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
 [py.warnings]
WARNING    /home/bogdanp/PhD/phd_venv/local/lib/python2.7/site-packages/numpy/core/numeric.py:482: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return array(a, dtype, copy=False, order=order)
 [py.warnings]


time slot 0
5. ms


In [13]:
spike_times[0]

[1.2000000000000002,
 5.2,
 5.4,
 5.800000000000001,
 7.0,
 7.3,
 10.0,
 10.5,
 10.6,
 11.1,
 11.600000000000001,
 11.799999999999999,
 12.100000000000001,
 12.200000000000001,
 14.100000000000001,
 15.5,
 16.0,
 16.099999999999998,
 16.400000000000002,
 16.8,
 16.900000000000002,
 17.5,
 17.9,
 18.8,
 19.1,
 19.4,
 1.2000000000000002,
 1.4,
 1.5,
 1.6,
 1.7000000000000002,
 2.8,
 3.7,
 4.800000000000001,
 4.8999999999999995,
 5.1000000000000005,
 7.9,
 9.1,
 9.3,
 9.600000000000001,
 10.3,
 11.799999999999999,
 12.100000000000001,
 12.4,
 12.6,
 12.700000000000001,
 12.8,
 13.9,
 15.1,
 15.8,
 15.9,
 16.400000000000002,
 16.5,
 17.1,
 18.4,
 18.8,
 19.1,
 19.4,
 19.700000000000003,
 0.0,
 0.5,
 3.2,
 4.2,
 4.8999999999999995,
 5.8999999999999995,
 6.5,
 6.6,
 6.7,
 8.700000000000001,
 12.100000000000001,
 12.700000000000001,
 13.4,
 13.8,
 14.4,
 14.6,
 15.1,
 15.2,
 17.400000000000002,
 18.2,
 18.3,
 18.500000000000004,
 19.4,
 0.4,
 2.0,
 2.1,
 2.7,
 3.3,
 4.7,
 5.800000000000001,
 

In [14]:
len(spike_times[0])

136

In [15]:
np.argsort(spike_times[0])

array([ 59,  82,  60,   0,  26,  27,  28,  29,  30,  83,  84,  85,  31,
        61,  86,  32,  62,  87,  33,  34,  63,  35,   1,   2,  88,   3,
        64,  65,  66,  67,   4,   5,  89,  36,  68,  90,  37,  38,  39,
        91,   6,  40,   7,   8,  92,   9,  10,  41,  11,  69,  12,  42,
        93,  13,  94,  43,  95,  44,  45,  70,  96,  46,  97,  71,  98,
        72,  99,  47, 100,  14,  73,  74, 101,  75,  48,  76,  15,  49,
        50,  16,  17, 102,  51,  18,  52, 103,  19,  20,  53,  77,  21,
        22, 104,  78,  79, 105,  54,  80,  23,  55, 106,  24,  56,  81,
        25,  57,  58, 123, 114, 124, 130, 131, 125, 115, 116, 132, 126,
       107, 108, 133, 117, 118, 134, 127, 109, 128, 119, 120, 135, 110,
       111, 121, 122, 129, 112, 113])